In [0]:
%sql
CREATE OR REPLACE FUNCTION mcp_002.default.fn_search_contracts(
  contractor_name STRING,   -- Optional. Pass NULL to skip. Case-insensitive partial match. Filters ContractorName.
  contract_status          STRING,   -- Optional. Pass NULL to skip. Case-insensitive exact match (normalized). Filters contract status.
  year_from              INT,       -- Optional. Pass NULL to skip. Filters Contract's AwardDate >= year_from.
  year_to                INT       -- Optional. Pass NULL to skip. Filters Contract's AwardDate <= year_to.
)
RETURNS TABLE (
  ContractID     BIGINT,
  ContractorID   BIGINT,
  ContractorName STRING,
  ContractCode   STRING,
  Title          STRING,
  AwardDate      DATE,
  TotalValue     DECIMAL(18,2),
  Currency       STRING,
  Status         STRING
)
COMMENT 'Search contracts. All parameters are optional. Pass NULL to ignore a parameter. contractor_name: case-insensitive partial match. status: case-insensitive exact match. year_from/year_to: numeric year filters on contract AwardDate.'
RETURN
SELECT
  c.ContractID,
  CAST(c.ContractorID AS BIGINT) AS ContractorID,
  ctr.ContractorName,
  c.ContractCode,
  c.Title,
  c.AwardDate,
  c.TotalValue,
  c.Currency,
  c.Status
FROM mcp_002.default.Contracts c JOIN mcp_002.default.Contractors ctr
  ON CAST(c.ContractorID AS BIGINT) = ctr.ContractorID
--WHERE c.Status = 'Active'
WHERE
  (
    contractor_name IS NULL OR contractor_name = ''
    OR LOWER(ctr.ContractorName) ILIKE CONCAT('%', regexp_replace(LOWER(contractor_name), '\\s+', '%'), '%')
  )
    AND (
        contract_status IS NULL OR
        -- Normalize both sides: lower, remove non-alphanumerics so 'On Hold' == 'onhold'
        REGEXP_REPLACE(LOWER(c.Status), '[^a-z0-9]', '') =
        REGEXP_REPLACE(LOWER(contract_status),   '[^a-z0-9]', '')
      )
  AND (year_from IS NULL OR year_from = 0 OR year(c.AwardDate) >= year_from)
  AND (year_to IS NULL OR year_to = 0 OR year(c.AwardDate) <= year_to);


In [0]:
%sql
CREATE OR REPLACE FUNCTION mcp_002.default.fn_total_value_by_contractor_and_status(
  contractor_name STRING,   -- Required. Case-insensitive partial match. Filters Contractor Name.
  contract_status STRING   -- Optional. Pass NULL to skip. Case-insensitive exact match (normalized). Filters contract status.
)
RETURNS TABLE ( TotalValue DECIMAL(18,2) )
COMMENT 'Searches by contractor. The contract status parameter is optional. Pass NULL to ignore status.  Return the total value of all contracts awarded to a specific contractor.'
RETURN
  -- COALESCE((
    SELECT SUM(c.TotalValue) AS TotalValue
    FROM mcp_002.default.Contracts   c
    JOIN mcp_002.default.Contractors ctr
      ON CAST(c.ContractorID AS BIGINT) = ctr.ContractorID
    WHERE LOWER(ctr.ContractorName) ILIKE CONCAT('%', regexp_replace(LOWER(contractor_name), '\\s+', '%'), '%')
    AND (
        contract_status IS NULL OR contract_status = '' OR
        -- Normalize both sides: lower, remove non-alphanumerics so 'On Hold' == 'onhold'
        REGEXP_REPLACE(LOWER(c.Status), '[^a-z0-9]', '') = REGEXP_REPLACE(LOWER(contract_status), '[^a-z0-9]', '')
      )
  -- ), 0);


In [0]:
%sql
CREATE OR REPLACE FUNCTION mcp_002.default.fn_orphan_contracts()
RETURNS TABLE (
  ContractID   BIGINT,
  ContractorID BIGINT,
  ContractCode STRING,
  Title        STRING,
  AwardDate    DATE,
  TotalValue   DECIMAL(18,2),
  Currency     STRING,
  Status       STRING
)
COMMENT 'Searches for orphaned, unassigned or abandoned contracts.'
RETURN
SELECT c.ContractID,
       CAST(c.ContractorID AS BIGINT) AS ContractorID,
       c.ContractCode, c.Title, c.AwardDate, c.TotalValue, c.Currency, c.Status
FROM mcp_002.default.Contracts c
LEFT ANTI JOIN mcp_002.default.Contractors ctr
  ON CAST(c.ContractorID AS BIGINT) = ctr.ContractorID;


In [0]:
%sql
CREATE OR REPLACE FUNCTION mcp_002.default.fn_contracts_by_status_and_range(
  status STRING, from_dt DATE, to_dt DATE
)
RETURNS TABLE (
  ContractID    BIGINT,
  ContractorID  BIGINT,
  ContractorName STRING,
  ContractCode  STRING,
  Title         STRING,
  AwardDate     DATE,
  TotalValue    DECIMAL(18,2),
  Currency      STRING,
  Status        STRING
)
RETURN
SELECT  c.ContractID,
        CAST(c.ContractorID AS BIGINT),
        ctr.ContractorName,
        c.ContractCode,
        c.Title,
        c.AwardDate,
        c.TotalValue,
        c.Currency,
        c.Status
FROM mcp_002.default.Contracts c
JOIN mcp_002.default.Contractors ctr
  ON CAST(c.ContractorID AS BIGINT) = ctr.ContractorID
WHERE c.Status = status
  AND c.AwardDate BETWEEN from_dt AND to_dt
ORDER BY c.AwardDate DESC;


In [0]:
%sql
CREATE OR REPLACE FUNCTION mcp_002.default.fn_contractor_totals()
RETURNS TABLE (
  ContractorID   BIGINT,
  ContractorName STRING,
  ContractCount  BIGINT,
  TotalValue     DECIMAL(38,2)
)
RETURN
SELECT  ctr.ContractorID,
        ctr.ContractorName,
        COUNT(c.ContractID)           AS ContractCount,
        COALESCE(SUM(c.TotalValue),0) AS TotalValue
FROM mcp_002.default.Contractors ctr
LEFT JOIN mcp_002.default.Contracts c
  ON CAST(c.ContractorID AS BIGINT) = ctr.ContractorID
GROUP BY ctr.ContractorID, ctr.ContractorName;


In [0]:
%sql
DROP FUNCTION IF EXISTS mcp_002.default.add_ten;

-- CREATE OR REPLACE FUNCTION mcp_002.default.add_ten(x INT)
-- RETURNS INT
-- RETURN x + 10;


In [0]:
%sql
-- # -- Table function, no parameters
-- # CREATE OR REPLACE FUNCTION mcp_002.default.get_status_codes()
-- # RETURNS TABLE (
-- #   statuscode STRING,
-- #   description STRING
-- # )
-- # RETURN
-- #   SELECT statuscode, description
-- #   FROM mcp_002.default.status_codes;


In [0]:
functions = spark.sql("SHOW FUNCTIONS IN mcp_002.default").collect()

# Filter out built-in functions (keep only those in your schema)
custom_funcs = [f.function for f in functions if f.function.startswith("mcp_002.default.")]

for fn in custom_funcs:
    print(f"DROP FUNCTION IF EXISTS {fn};")


In [0]:
%sql
DROP FUNCTION IF EXISTS mcp_002.default.fn_total_contract_value_by_contractor_and_status;